In [2]:
# import necessary libraries

import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
#import glove
#from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [9]:
# Helper functions for training the multimodal model

def reset_keras(model):
    """reset keras session"""
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def create_dataset(dict_of_ner):
    """create the dataset"""
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    """make model predictions"""
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth,                          
                          embed_name, problem_type, iteration, hidden_unit_size,                         
                          sequence_name, type_of_ner):
    """save metrics of model"""
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1
    
    result_path = "results/multimodal"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-avg-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):
    """define the model specifications"""

    #if embedding_name == "concat":
    #    input_dimension = 200
    #else:
    #    input_dimension = 100
    input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
    logits_regularizer = keras.regularizers.l2(0.01)
    
    #preds = Dense(1, activation='sigmoid',use_bias=False,
    #                     kernel_initializer=tf.contrib.layers.xavier_initializer(), 
    #              kernel_regularizer=logits_regularizer)(x)
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.GlorotUniform(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    opt = Adam(lr=0.001, decay = 0.01)
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model

In [5]:
# import dataset

type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

In [55]:
# Train and test the multimodal model for each embedding type and target problem

embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']


num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
batch_size = 64
iter_num = 2
unit_sizes = [128, 256]
#unit_sizes = [256]

#layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
            print ("Embedding: ", embed_name)
            print("=============================")

            temp_train_ner = dict((k, ner_word2vec[k]) for k in train_ids)
            temp_dev_ner = dict((k, ner_word2vec[k]) for k in dev_ids)
            temp_test_ner = dict((k, ner_word2vec[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)


            for iteration in range(1, iter_num):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.keras"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                        save_best_only=True, mode='min')


                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)
                    
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, 
                              batch_size=batch_size )

                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                                embed_name, each_problem, iteration, each_unit_size, 
                                each_layer, type_of_ner)
                    
                    reset_keras(model)
                    #del model
                    clear_session()
                    gc.collect()

Layer:  GRU
Hidden unit:  256
Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
244/244 [==============================] - ETA: 0s - loss: 0.2842 - acc: 0.9058
Epoch 1: val_loss improved from inf to 0.24494, saving model to avg-word2vec-mort_hosp-best_model.keras
244/244 [==============================] - 28s 100ms/step - loss: 0.2842 - acc: 0.9058 - val_loss: 0.2449 - val_acc: 0.9161
Epoch 2/100
243/244 [============================>.] - ETA: 0s - loss: 0.2444 - acc: 0.9154
Epoch 2: val_loss improved from 0.24494 to 0.24015, saving model to avg-word2vec-mort_hosp-best_model.keras
244/244 [==============================] - 21s 85ms/step - loss: 0.2443 - acc: 0.9155 - val_loss: 0.2402 - val_acc: 0.9188
Epoch 3/100
244/244 [==============================] - ETA: 0s - loss: 0.2294 - acc: 0.9201
Epoch 3: val_loss improved from 0.24015 to 0.23655, saving model to avg-word2vec-mort_hosp-best_model.keras
244/244 [==============================] 

#### Show average over iterations for each category/embedding metric

In [56]:
import os
import pickle
import numpy as np
import pandas as pd

# Define categories and metrics
categories = ["GRU-128", "GRU-256"]
metrics = {"auc":"AUROC", "auprc":"AUPRC", "acc":"Accuracy", "F1":"F1"}
tasks = ["mort_hosp", "mort_icu", "los_3", "los_7"]
embeddings = ["word2vec", "fasttext", "concat"]

# Initialize dictionaries to store results
results = {category: {embedding: {task: {metric_name: [] for metric, metric_name in metrics.items()} for task in tasks} for embedding in embeddings} for category in categories}

# Directory where pickle files are stored
directory = "results/multimodal/"

# Loop through each file
for filename in os.listdir(directory):
    if filename.endswith("-new-avg-.p"):
        parts = filename.split("-")
        category = parts[0] + "-" + parts[1]
        embedding = parts[2]
        task = parts[3]
        if category in categories and task in tasks and embedding in embeddings:
            result_dict = pd.read_pickle(os.path.join(directory, filename))
            for metric, metric_name in metrics.items():
                results[category][embedding][task][metric_name].append(result_dict[metric])

# Calculate average and standard deviation
for category in categories:
    print(f"Category: {category}")
    for embedding in embeddings:
        print(f"Embedding: {embedding}")
        df_data = {task: {} for task in tasks}
        for task in tasks:
            task_data = {}
            for metric, metric_name in metrics.items():
                values = results[category][embedding][task][metric_name]
                mean = np.mean(values)
                std = np.std(values)
                task_data[metric_name] = f"{mean:.4f} \u00B1 {std:.4f}"
            df_data[task] = task_data
        df = pd.DataFrame(df_data).transpose()
        print(df)
        print()

Category: GRU-128
Embedding: word2vec
                     AUROC            AUPRC         Accuracy               F1
mort_hosp  0.8886 ± 0.0000  0.5962 ± 0.0000  0.9192 ± 0.0000  0.4863 ± 0.0000
mort_icu   0.8909 ± 0.0000  0.5332 ± 0.0000  0.9410 ± 0.0000  0.4505 ± 0.0000
los_3      0.7048 ± 0.0000  0.6456 ± 0.0000  0.6656 ± 0.0000  0.5617 ± 0.0000
los_7      0.7356 ± 0.0000  0.2229 ± 0.0000  0.9195 ± 0.0000  0.0532 ± 0.0000

Embedding: fasttext
                     AUROC            AUPRC         Accuracy               F1
mort_hosp  0.8844 ± 0.0000  0.5844 ± 0.0000  0.9188 ± 0.0000  0.4774 ± 0.0000
mort_icu   0.8954 ± 0.0000  0.5430 ± 0.0000  0.9441 ± 0.0000  0.4711 ± 0.0000
los_3      0.7042 ± 0.0000  0.6407 ± 0.0000  0.6609 ± 0.0000  0.5553 ± 0.0000
los_7      0.7367 ± 0.0000  0.2166 ± 0.0000  0.9195 ± 0.0000  0.0378 ± 0.0000

Embedding: concat
                     AUROC            AUPRC         Accuracy               F1
mort_hosp  0.8842 ± 0.0000  0.5911 ± 0.0000  0.9183 ± 0.0000  0.